In [9]:
import json
import os
import pandas as pd
from pycocotools.coco import COCO

In [128]:
# submission 양식에 맞게 output 후처리
def json_to_submission(json_file_path, submission_path, output_path, output_file_name, score_thr,test=True):

    with open(json_file_path, 'r') as f:
        data = json.load(f)

    coco = COCO(submission_path)
    img_ids = coco.getImgIds()

    prediction_strings = ['' for _ in range(len(img_ids))]
    if test:
        file_names = [f'test/{str(img_id).zfill(4)}.jpg' for img_id in range(len(img_ids))]
    else:
        file_names = [f'train/{str(img_id).zfill(4)}.jpg' for img_id in range(len(img_ids))]
    for info in data:

        img_id = info['image_id']
        xmin, ymin, w, h = info['bbox']
        xmax, ymax = xmin+w, ymin+h
        score = info['score']
        cate_id = info['category_id']

        if score < score_thr:
            continue

        prediction_list = map(str, [cate_id, score, xmin, ymin, xmax, ymax])
        prediction_string = ' '.join(prediction_list) + ' '

        prediction_strings[img_id] += prediction_string


    submission = pd.DataFrame()
    submission['PredictionString'] = prediction_strings
    submission['image_id'] = file_names

    submission.to_csv(os.path.join(output_path, output_file_name), index=None)
    print(submission.head())
    print(submission.tail())

In [133]:
json_file_path = './mmdetection/work_dirs/ddq_recycle/result/ddq_train_tta.bbox.json'
submission_path = './dataset/train.json'
output_path = './submission'
output_file_name = 'ddq_result_train_tta_0.05.csv'
score_thr = 0.05

json_to_submission(json_file_path, submission_path, output_path, output_file_name, score_thr, test=False)

loading annotations into memory...
Done (t=0.09s)
creating index...
index created!
                                    PredictionString        image_id
0  0 0.8503260016441345 200.0750732421875 194.718...  train/0000.jpg
1  4 0.7346956729888916 730.5899047851562 314.497...  train/0001.jpg
2  3 0.8661016225814819 269.3987731933594 165.407...  train/0002.jpg
3  2 0.8895310759544373 463.60284423828125 368.44...  train/0003.jpg
4  1 0.7084575891494751 481.79339599609375 464.47...  train/0004.jpg
                                       PredictionString        image_id
4878  0 0.8422749042510986 380.36224365234375 370.90...  train/4878.jpg
4879  7 0.8381984829902649 483.37432861328125 640.56...  train/4879.jpg
4880  0 0.8522360324859619 198.9588623046875 212.132...  train/4880.jpg
4881  7 0.8725916147232056 153.0755615234375 469.776...  train/4881.jpg
4882  5 0.8223598003387451 5.93878173828125 124.9717...  train/4882.jpg
